In [ ]:
import os
import itertools
import json
import pandas as pd
import numpy as np

data_root="./data"

with open(os.path.join(data_root, "train/attr_to_attrvals.json")) as f:
    atts = json.load(f)
    for k in atts:
        att = {}
        for i, vi in enumerate(sorted(atts[k])):
            for vii in vi.split("="):
                att[vii] = i
        atts[k] = att
        
def get_v(t, k):
    if (k == "裤门襟" and "裤" not in t) or (k == "闭合方式" and ("鞋" not in t and "靴" not in t)):
        return ""
    mv = ""
    for v in atts[k]:
        if v in t and len(v) > len(mv):
            mv = v
    return mv

def v2id(k, v):
    if v == "": return -1
    return atts[k][v]

all_values = [v.keys() for v in atts.values()]
all_values = list(itertools.chain.from_iterable(all_values))
all_values.extend([ "常规厚度", "厚度常规", "加绒", "加绒裤", "白鸭绒", "2016年", "2017年", "2018年", "2019年", "2020年", "2021年", "2022年"])
print(all_values)
with open(os.path.join(data_root, "train/jieba.dict"), "w", encoding="UTF-8") as f:
    for line in all_values:
        f.write(line + "\n")

In [ ]:
text_df = []; feat = []
with open(os.path.join(data_root, "train/train_fine.txt")) as f:
    for l in f:
        l = json.loads(l.strip())
        img_idx = len(feat)
        feat.append(l.pop("feature"))
        text_df.append([l["title"], img_idx, l["match"]["图文"], [], []])
        for k in l["key_attr"]:
            text_df[-1][-2].append(k)
            text_df[-1][-1].append(l["key_attr"][k])
        if l["match"]["图文"]:
            attrs_remain = [k for k in atts if k not in l["key_attr"]]
            for k in attrs_remain:
                v = get_v(l["title"], k)
                if v:
                    text_df[-1][-2].append(k)
                    text_df[-1][-1].append(v)

            

with open(os.path.join(data_root, "train/train_coarse.txt")) as f:
    for l in f:
        l = json.loads(l.strip())
        img_idx = len(feat)
        feat.append(l.pop("feature"))
        text_df.append([l["title"], img_idx, l["match"]["图文"], [], []])
        if l["match"]["图文"]:
            for k in atts:
                v = get_v(l["title"], k)
                if v:
                    text_df[-1][-2].append(k)
                    text_df[-1][-1].append(v)

In [ ]:
text_df = pd.DataFrame(text_df)
text_df.columns = ["text", "img_idx", "label", "keys", "values"]
text_df.to_csv(os.path.join(data_root, "train/train_text.csv"), index = False)
np.save(os.path.join(data_root, "train/train.npy"), np.stack(feat))

In [ ]:
text_df = []; feat = []; name2id = []
with open(os.path.join(data_root, "preliminary_testB.txt")) as f:
    for l in f:
        l = json.loads(l.strip())
        img_idx = len(feat)
        feat.append(l.pop("feature"))
        name2id.append([l['img_name'], img_idx])
        text_df.append([l["title"], img_idx, 0, [], []])
        for k in l["query"]:
            if k == "图文": continue
            v = get_v(l["title"], k)
            text_df[-1][-2].append(k)
            text_df[-1][-1].append(v)

In [ ]:
text_df = pd.DataFrame(text_df)
text_df.columns = ["text", "img_idx", "label", "keys", "values"]
text_df.to_csv(os.path.join(data_root, "testB_text.csv"), index = False)
name2id = pd.DataFrame(name2id)
name2id.columns = ["img_name", "img_idx"]
name2id.to_csv(os.path.join(data_root, "name_to_id_B.csv"), index = False)
np.save(os.path.join(data_root, "testB.npy"), np.stack(feat))